In [1]:
#ROP Consolidation of Data Files

#Outline
#1. Load all data files
#2. No trimming. No missing data.
#3. Add date and absolute time to each row
#4. Include all the data that you have

In [2]:
%matplotlib inline

#Import dependencies
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
#Load ROPtimes.py, which contains a dict per subject and the datetime values of which eyedrops and eye exam occured.
import ROPtimes as rt 

In [3]:
#Code block full of different lists
# Baby_lst is the list of all the patients with data files in the study, Moberg files excluded.
# TCv1 is the list of patients with code accounting for the time correction 
    # This was before the time sync issue between machines was discovered. Time differences was the same in all subjects.
# RoomAir is the list of subjects belonging to the room air group.
# Moberg is the list of subjects that was analyzed using the Moberg machine (https://www.moberg.com/products/cns-monitor)

Baby_lst = [
    'ROP003', 'ROP005', 'ROP006', 'ROP007', 'ROP008', 
    'ROP009', 'ROP010', 'ROP011', 'ROP012', 'ROP013', 
    'ROP014', 'ROP015', 'ROP017', 'ROP018', 'ROP019', 
    'ROP020', 'ROP021', 'ROP022', 'ROP023', 'ROP025', 
    'ROP026', 'ROP027', 'ROP028', 'ROP029', 'ROP030', 
    'ROP031', 'ROP032', 'ROP033', 'ROP034', 'ROP035', 
    'ROP036', 'ROP037', 'ROP039', 'ROP042', 'ROP043',
    'ROP044', 'ROP045',
    ] #exclude ones with Moberg

TCv1 = [
    'ROP003', 'ROP005', 'ROP006', 'ROP007', 'ROP008', 
    'ROP009', 'ROP010', 'ROP011', 'ROP012', 'ROP013', 
    'ROP014', 'ROP015', 'ROP017', 'ROP018', 'ROP019', 
    'ROP020', 'ROP021', 'ROP022', 'ROP023', 'ROP025', 
    'ROP026', 'ROP027',
    ]

RoomAir = [
    'ROP005', 'ROP007', 'ROP018', 'ROP019', 'ROP021', 
    'ROP023', 'ROP024', 'ROP026', 'ROP029', 'ROP030',
    'ROP031', 'ROP032', 'ROP033', 'ROP034', 'ROP041', 
    'ROP042', 'ROP043', 'ROP044', 'ROP045'
    ]

Moberg = [
    'ROP038', 'ROP040', 'ROP041',
    ]

In [4]:
#This code block creates a function that does the following:
    #Load the files relevant to each subject.
    #Correct time difference if they are in the TCv1 list (ROP#029 had its own time difference values)
    #Accounts for missing machines, if any.
    #Adds a "Phase" Column to describe the data point in context of the ROP exam
        #either Baseline, Eyedrop1, Eyedrop2, Eyedrop3, During, or Post
    

def loadandtransformfile(Baby):
    
    #add all times to dict
    BabyDict = getattr(rt, Baby)

    print '\n %s' % Baby
    
    #load all files
    try:
        dfNIRSpre0 = pd.read_csv('C:\Users\John\Documents\GitHub\dissertation\CleanData\NIRS\\' + Baby + 'NIRS.csv',
                             parse_dates={'timestamp': ['Date',' Time']},
                             index_col='timestamp',
                             usecols=['Date', ' Time', ' Ch2 %StO2'],
                             na_values=['0'])
    except:
        dfNIRSpre0 = pd.read_csv('C:\Users\John\Documents\GitHub\dissertation\CleanData\NIRS\\' + Baby + 'NIRS.csv',
                             parse_dates={'timestamp': ['Date','Time']},
                             index_col='timestamp',
                             usecols=['Date', 'Time', ' Ch2 %StO2'],
                             na_values=['0'])
        

    dfPOpre = pd.read_csv('C:\Users\John\Documents\GitHub\dissertation\CleanData\Pulse Ox\\' + Baby + 'PO.csv',
                       parse_dates={'timestamp': ['Date','Time']},
                       index_col='timestamp',
                       usecols=['Date', 'Time', 'SpO2', 'PR'],
                       na_values=['0'], sep=',')

    dfNIRSpre = dfNIRSpre0.rename(columns={' Ch2 %StO2': 'StO2'}) #rename NIRS columns
    
    #This code is to make the combined dataframe come in even seconds.
    #The corrected is 1 second, + for NIRS and - for PO.

    TCcorrect = timedelta(seconds=1)

    ncorr = dfNIRSpre.index+TCcorrect
    pcorr = dfPOpre.index-TCcorrect

    if dfNIRSpre.index[:1].second % 2 == 0:
        if dfPOpre.index[:1].second % 2 == 0:
            print 'Both NIRS and PO indices are even.'
        elif dfPOpre.index[:1].second % 2 != 0:
            print 'NIRS even, PO odd. PO index corrected.'
            dfPOpre = dfPOpre.set_index(pcorr)
        else:
            raise NameError('Indices are messed up')
    elif dfNIRSpre.index[:1].second % 2 != 0:
        if dfPOpre.index[:1].second % 2 == 0:
            print 'NIRS odd, PO even. NIRS index corrected.'
            dfNIRSpre = dfNIRSpre.set_index(ncorr)
        elif dfPOpre.index[:1].second % 2 != 0:
            print 'Both NIRS and PO indices are odd. Both corrected'
            dfNIRSpre = dfNIRSpre.set_index(ncorr)
            dfPOpre = dfPOpre.set_index(pcorr)
    else:
        raise NameError('Indices are messed up')
    
    if Baby in TCv1:
        TCnirs = timedelta(minutes=2, seconds=10)
        TCpo = timedelta(minutes=1, seconds=32)
        # NIRS is slower than correct time, need to add TCnirs to catch it up
        # PO is faster than correct time, need to subtract TCpo to slow it down

        dfNIRS = dfNIRSpre.set_index([dfNIRSpre.index+TCnirs]) #NIRS Time Correction
        dfPO = dfPOpre.set_index([dfPOpre.index-TCpo]) #PO Time Correction
    
    elif Baby == 'ROP029':
        TC = timedelta(minutes=5)
        dfNIRS = dfNIRSpre.set_index([dfNIRSpre.index-TC]) #NIRS Time Correction
        dfPO = dfPOpre.set_index([dfPOpre.index-TC]) #PO Time Correction
        
    else:
        TC = timedelta(minutes=0)
        dfNIRS = dfNIRSpre.set_index([dfNIRSpre.index-TC]) #NIRS Time Correction
        dfPO = dfPOpre.set_index([dfPOpre.index-TC]) #PO Time Correction
    
    #for babies that only had one machine

    dffakePO = pd.DataFrame({'SpO2':0, 'PR':0}, index=dfNIRS.index)
    dffakeNIRS = pd.DataFrame({'StO2':0}, index=dfPO.index)

    if len(dfNIRS) > 5:
        if len(dfPO) > 5:
            df = dfNIRS.combine_first(dfPO) #Combine two DataFrame objects and default to non-null values in frame
            Masimo = True
            NIRS = True
            print 'Both machines on'
        elif len(dfPO) < 5:
            df = dfNIRS.combine_first(dffakePO)
            print 'Only NIRS on'
            NIRS = True
            Masimo = False
    elif len(dfNIRS) < 5:
        df = dffakeNIRS.combine_first(dfPO)
        Masimo = True
        NIRS = False
        print 'Only Masimo on'
    else:
        raise NameError('Check your files')
    
    #add new columns
    #df['Date'] = [d.date() for d in df.index]
    #df['Time'] = [d.time() for d in df.index]
    
    df['ID'] = Baby[4:]
    df.loc[:BabyDict['EyeDrop1'], 'Phase'] = 'PRE'
    df.loc[BabyDict['EyeDrop1']:BabyDict['EyeDrop2'], 'Phase'] = 'ED1'
    df.loc[BabyDict['EyeDrop2']:BabyDict['EyeDrop3'], 'Phase'] = 'ED2'
    df.loc[BabyDict['EyeDrop3']:BabyDict['ExamStart'], 'Phase'] = 'ED3'
    df.loc[BabyDict['ExamStart']:BabyDict['ExamEnd'], 'Phase'] = 'DURING'
    df.loc[BabyDict['ExamEnd']:, 'Phase'] = 'AFTER'
    #df.loc[BabyDict['ExamStart']-timedelta(minutes=2), 'Phase'] = 'Sweet-Ease'
    
    if Baby in RoomAir: 
        df['RoomAir'] = 'YES'
    else:
        df['RoomAir'] = 'NO'
   
    if Masimo != False and NIRS != False:
        dfpreFTOE1 = pd.DataFrame({'SpO2' : df['SpO2'].values, 'StO2' : df['StO2'].values}, index=df.index)
        dfpreFTOE1 = dfpreFTOE1.dropna(how='any')

        psa1 = dfpreFTOE1['SpO2']
        prs1 = dfpreFTOE1['StO2']

        dfFTOE1 = pd.DataFrame({'FTOE' : (((psa1-prs1)/psa1)*100)})
        #dfFTOE = dfFTOE.round(decimals=3)
        
        df = df.join(dfFTOE1)
        
    if Masimo == False:
        df['PR'] = np.NaN
        df['SpO2'] = np.NaN
        df['FTOE'] = np.NaN
    elif NIRS == False:
        df['StO2'] = np.NaN
        df['FTOE'] = np.NaN
    else:
        df = df
    
    df = df[['ID', 'RoomAir', 'Phase', 'PR', 'SpO2', 'StO2', 'FTOE']]
    df=df.rename(columns = {'PR':'HR'})

    return df

In [5]:
#Code block used to test individual subjects
    #Use this block to test new subjects to see if raw data is clean
    
testdf = loadandtransformfile('ROP044')


 ROP044
NIRS odd, PO even. NIRS index corrected.
Both machines on


In [6]:
testdf.index

DatetimeIndex(['2017-11-07 23:11:02', '2017-11-07 23:11:04',
               '2017-11-07 23:11:06', '2017-11-07 23:11:08',
               '2017-11-07 23:11:10', '2017-11-07 23:11:12',
               '2017-11-07 23:11:14', '2017-11-07 23:11:16',
               '2017-11-07 23:11:18', '2017-11-07 23:11:20',
               ...
               '2017-11-10 15:30:27', '2017-11-10 15:30:29',
               '2017-11-10 15:30:31', '2017-11-10 15:30:33',
               '2017-11-10 15:30:35', '2017-11-10 15:30:37',
               '2017-11-10 15:30:39', '2017-11-10 15:30:41',
               '2017-11-10 15:30:43', '2017-11-10 15:30:45'],
              dtype='datetime64[ns]', name=u'timestamp', length=45802, freq=None)

In [7]:
#This code block creates a function to load Moberg CNS Monitor output files.

def Moberg_loadandxformfile(Baby):
    
    df = pd.read_csv('C:\Users\John\Documents\GitHub\dissertation\CleanData\Moberg\\' + Baby + '.csv',
                             index_col=0,
                             usecols=['Unnamed: 0', 'HR,na,IntelliVue(bpm)', 'SpO2,na,IntelliVue(%)', 'StO2,1,FORESIGHT(%)'],
                             na_values=['0'], parse_dates=True)
    df['ID'] = Baby[4:]
    BabyDict = getattr(rt, Baby)

    print '\n %s' % Baby
    
    df.loc[:BabyDict['EyeDrop1'], 'Phase'] = 'PRE'
    df.loc[BabyDict['EyeDrop1']:BabyDict['EyeDrop2'], 'Phase'] = 'ED1'
    df.loc[BabyDict['EyeDrop2']:BabyDict['EyeDrop3'], 'Phase'] = 'ED2'
    df.loc[BabyDict['EyeDrop3']:BabyDict['ExamStart'], 'Phase'] = 'ED3'
    df.loc[BabyDict['ExamStart']:BabyDict['ExamEnd'], 'Phase'] = 'DURING'
    df.loc[BabyDict['ExamEnd']:, 'Phase'] = 'AFTER'

    if Baby in RoomAir: 
        df['RoomAir'] = 'YES'
    else:
        df['RoomAir'] = 'NO'

    df.rename(columns={'HR,na,IntelliVue(bpm)': 'HR',
                       'SpO2,na,IntelliVue(%)': 'SpO2',
                       'StO2,1,FORESIGHT(%)': 'StO2'}, inplace=True)

    df['FTOE'] = ((df['SpO2']-df['StO2'])/(df['SpO2'])*100)
    
    df[['HR']] = df[['HR']].apply(pd.to_numeric)
    
    print "Moberg File Read Success"
    
    return df

In [8]:
#Code block to test if Moberg load function works
testdf2 = Moberg_loadandxformfile('ROP038')


 ROP038
Moberg File Read Success


In [9]:
%%time

# This code block batch runs forloops processes all available ROP exam data files to 
    # consolidate it all into one list of dataframes (dflst) to be later concatenated into one CSV file.
    # Managing indivudal data files into one file allows for easier downstream data wrangling.
    # %%time is a magic function used in Jupyter to measure how long it takes to run this code block.
    # Warning: May take a long time


dflst = [] #List of all subject dataframes

for i in Baby_lst:
    df1 = loadandtransformfile(i)
    dflst.append(df1)

for i in Moberg:
    df2 = Moberg_loadandxformfile(i)
    dflst.append(df2)


 ROP003
Both NIRS and PO indices are odd. Both corrected
Only NIRS on

 ROP005
NIRS even, PO odd. PO index corrected.
Only NIRS on

 ROP006
Both NIRS and PO indices are odd. Both corrected
Only Masimo on

 ROP007
NIRS even, PO odd. PO index corrected.
Both machines on

 ROP008
Both NIRS and PO indices are odd. Both corrected
Both machines on

 ROP009
NIRS odd, PO even. NIRS index corrected.
Both machines on

 ROP010
Both NIRS and PO indices are even.
Only Masimo on

 ROP011
NIRS even, PO odd. PO index corrected.
Both machines on

 ROP012
NIRS even, PO odd. PO index corrected.
Both machines on

 ROP013
NIRS odd, PO even. NIRS index corrected.
Both machines on

 ROP014
Both NIRS and PO indices are odd. Both corrected
Both machines on

 ROP015
Both NIRS and PO indices are even.
Both machines on

 ROP017
Both NIRS and PO indices are even.
Both machines on

 ROP018
NIRS odd, PO even. NIRS index corrected.
Both machines on

 ROP019
NIRS even, PO odd. PO index corrected.
Both machines on

 R

In [10]:
#Combine all the dataframes into one dataframe
df_comb = pd.concat(dflst)

In [11]:
#Save it into one CSV file
df_comb.to_csv('ROP_alldata.csv')

In [13]:
df_comb.index

DatetimeIndex(['2015-05-20 08:45:36', '2015-05-20 08:45:38',
               '2015-05-20 08:45:40', '2015-05-20 08:45:42',
               '2015-05-20 08:45:44', '2015-05-20 08:45:46',
               '2015-05-20 08:45:48', '2015-05-20 08:45:50',
               '2015-05-20 08:45:52', '2015-05-20 08:45:54',
               ...
               '2017-10-27 14:50:06', '2017-10-27 14:50:08',
               '2017-10-27 14:50:10', '2017-10-27 14:50:12',
               '2017-10-27 14:50:14', '2017-10-27 14:50:16',
               '2017-10-27 14:50:18', '2017-10-27 14:50:20',
               '2017-10-27 14:50:22', '2017-10-27 14:50:24'],
              dtype='datetime64[ns]', length=2630992, freq=None)